В данном блоке сделаем предобработку данных для дальнейшего анализа.  
Добавим спарсенные новые данные, заполним пропуски.  
Приведем в удобный для работы вид данных и т.д.

In [460]:
import pandas as pd
import numpy as np
import re
import time
import datetime



import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\skins\AppData\Roaming\nltk_data...


In [431]:
# Сделаем сразу единую таблицу, для работы (чтобы с ней работать над пропусками, новыми фичами и т.д.)

# вот так можно бороться с тем, что просто скачивание из Google Drive по обычной ссылке не получается
url_main='https://drive.google.com/file/d/1bfGQ_OVPj1N3TKdW4l6a78CQFihEyYhp/view?usp=sharing'
url_kaggle= 'https://drive.google.com/file/d/1xU6EZs5BeMRfyt8hp1uNpZBy2JCKn7w7/view?usp=sharing'

url_main='https://drive.google.com/uc?id=' + url_main.split('/')[-2]
url_kaggle='https://drive.google.com/uc?id=' + url_kaggle.split('/')[-2]

data_main = pd.read_csv(url_main)
data_kaggle = pd.read_csv(url_kaggle)

# Добавим признак 'Main' для отделения основной выборки от валидационной
data_main['Main'] = True
data_kaggle['Main'] = False

# объединим в одну табличку
data = pd.concat([data_main, data_kaggle])

# добавим столбец с полной ссылкой на ресторан
data['URL_TA_full'] = 'https://www.tripadvisor.com' + data['URL_TA']

parsed_data = pd.read_csv(r'parsed.csv')

In [432]:
# Посмотрим размер данных
data.shape

(50000, 12)

In [433]:
# Посмотрим размер спарсенных данных
parsed_data.shape

(49963, 14)

37 ресторанов в исходных данных являются дубликатами, как правило, незначительно отличается ranking.
Причем они перемешаны, некоторые находятся только в основных данных, некоторые в данных для теста


In [434]:
data.columns

Index(['Restaurant_id', 'City', 'Cuisine Style', 'Ranking', 'Rating',
       'Price Range', 'Number of Reviews', 'Reviews', 'URL_TA', 'ID_TA',
       'Main', 'URL_TA_full'],
      dtype='object')

In [435]:
parsed_data.columns

Index(['Unnamed: 0', 'alive', 'price_range', 'cuisines', 'reviews', 'rating',
       'mean_check', 'excelent', 'very_good', 'average', 'poor', 'terrible',
       'ranking', 'URL_TA_full'],
      dtype='object')

In [436]:
df = data.merge(parsed_data, on=['URL_TA_full'])

In [437]:
df.shape

(50000, 25)

In [438]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      50000 non-null  object 
 1   City               50000 non-null  object 
 2   Cuisine Style      38410 non-null  object 
 3   Ranking            50000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        32639 non-null  object 
 6   Number of Reviews  46800 non-null  float64
 7   Reviews            49998 non-null  object 
 8   URL_TA             50000 non-null  object 
 9   ID_TA              50000 non-null  object 
 10  Main               50000 non-null  bool   
 11  URL_TA_full        50000 non-null  object 
 12  Unnamed: 0         50000 non-null  int64  
 13  alive              50000 non-null  int64  
 14  price_range        34133 non-null  object 
 15  cuisines           36647 non-null  object 
 16  reviews            365

In [439]:
# Поработаем с пропусками. Для начала заполним пропуски данными, которые получилось спарсить с сайта Трипэдвайзер
def fill_nan(row_base, row_parsed):
    for i in range(len(row_base)):
        try:
            if np.isnan(row_base.iloc[i]):
                row_base.iloc[i] = row_parsed.iloc[i]
        except:
            if row_base.iloc[i] == '[]':
                row_base.iloc[i] = row_parsed.iloc[i]
            else:
                continue


In [440]:
fill_nan(df['Cuisine Style'], df['cuisines'])
fill_nan(df['Price Range'], df['price_range'])
fill_nan(df['Number of Reviews'], df['reviews'])

In [441]:
# Картина с пропусками существенно улучшилась
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      50000 non-null  object 
 1   City               50000 non-null  object 
 2   Cuisine Style      46803 non-null  object 
 3   Ranking            50000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        43160 non-null  object 
 6   Number of Reviews  49157 non-null  object 
 7   Reviews            49998 non-null  object 
 8   URL_TA             50000 non-null  object 
 9   ID_TA              50000 non-null  object 
 10  Main               50000 non-null  bool   
 11  URL_TA_full        50000 non-null  object 
 12  Unnamed: 0         50000 non-null  int64  
 13  alive              50000 non-null  int64  
 14  price_range        34133 non-null  object 
 15  cuisines           36647 non-null  object 
 16  reviews            365

In [442]:
# кстати ради интереса можно посмотреть, сколько рейтингов, которые нужно предсказать, мы спарсили с сайта Трипэдвайзера )))
df[df['Main'] == False]['rating'].info()

<class 'pandas.core.series.Series'>
Int64Index: 10000 entries, 3441 to 49999
Series name: rating
Non-Null Count  Dtype 
--------------  ----- 
7373 non-null   object
dtypes: object(1)
memory usage: 156.2+ KB


In [443]:
# сохраним себе отдельно спарсенный рейтинг
parsed_raiting = df[df['Main'] == False][['Restaurant_id', 'rating']]
parsed_raiting.to_csv(r'parsed_raiting.csv')

как видим по 7373 ресторанам можно не делать предсказание рейтинга, так как он у нас уже есть )))

In [444]:
# остатки пропусков в колонке Cuisine Style заполним каким-нибудь одним стилем - типа 'usual'
# так же переведем формат данных в удобный список
df['Cuisine Style'] = df['Cuisine Style'].fillna("['usual']")
df['Cuisine Style'] = df['Cuisine Style'].apply(lambda x: "['usual']" if x == '[]' else x)
df['Cuisine Style'] = df['Cuisine Style'].apply(lambda x: x[2:-2].split("', '"))

In [445]:
# В Price Range много пропусков, обусловленно тем, что либо ресторан уже неживой, либо на сайте Трипэдвайзера этих данных нет.
# выведем все пропуски в отдельную категорию '$*'
df['Price Range'] = df['Price Range'].apply(lambda x: x if x in ['$$ - $$$', '$$$$', '$'] else '$*')
df['Price Range'].unique()

array(['$$ - $$$', '$$$$', '$*', '$'], dtype=object)

In [446]:
# Number Reviews - кол-во отзывов. Логично Nan заменить на 0.
df['Number of Reviews'].fillna(0, inplace=True)

In [447]:
# спарсенный рейтинг приведем к float, пропуски заменим (0, средним или медианой...) - медианой
rating_median = df[df['Main'] == True]['Rating'].median()
df['rating'] = df['rating'].apply(lambda x: float(x) if type(x) == str else rating_median)

In [448]:
# Reviews Содержит как текст отзыва, так и даты когда были опубликованы эти отзывы
# Выдернем отдельно даты, отдельно текст отзыва

# добавим разницу в днях между отзывами и время в днях с последнего отзыва 
#сначала новые столбцы с датами отзывов
pattern = re.compile('\d{2}/\d{2}/\d{4}')
reviews=[]
for i in df['Reviews']:
    try:
        reviews.append(re.findall(pattern, i))
    except:
        reviews.append(['01/01/1970', '01/01/1970'])

date=pd.DataFrame(reviews)  #отдельный датафрейм для удобства
date = date.fillna(0)  # заполним пропуски нулями
date.columns=['date1', 'date2']
date['date1'] = pd.to_datetime(date['date1'])
date['date2'] = pd.to_datetime(date['date2']) 

df['time_between'] = (date['date1'] - date['date2']).apply(lambda x : abs(float(x.days)))  
# время от последнего отзыва

last_time_review = []
for i in date['date1']:
    last_time_review.append(float((datetime.datetime.today() - i).days))
    
df['last_time_review'] = last_time_review

# Текст отзыва вынесем в отдельный столбец, после чего сам Reviews нам будет не нужен
df['text_reviews'] = df['Reviews'].str.slice(start = 2)
df['text_reviews'] = df['text_reviews'].str.split('],').str[0]
# пропуски заменим на ''
df['text_reviews'].fillna('', inplace=True)

In [449]:
# Оценки отзывов (excelent, very_good, average, poor, terrible). Пропуски заполним нулями, цифры приведем к float
for i in ['excelent', 'very_good', 'average', 'poor', 'terrible']:
    df[i].fillna('0', inplace=True)
    df[i] = df[i].apply(lambda x: x.replace(',', '') if type(x)==str else x)
    df[i] = df[i].apply(lambda x: float(x))

In [450]:
# ranking указано место среди ресторанов города (по определенному виду кухни, либо в целом)
# уберем запятые внутри цифр
df['ranking'] = df['ranking'].apply(lambda x: x.replace(',', '') if type(x)==str else x)
# выдернем цифры из ячеек
pattern = re.compile(r'\d+')
ranks=[]
for i in df['ranking']:
    try:
        ranks.append(re.findall(pattern, i))
    except:
        ranks.append([0,0])
df['ranking'] = ranks
df['ranking'] = df['ranking'].apply(lambda x: [0,0] if x == [] else x)

#добавим кол-во ресторанов в городе
df['rest_number'] = df['ranking'].apply(lambda x: float(x[-1]))

# Заполним кол-во ресторанов их макс значением по городу
data_grouped = df.groupby('City').max()
for i in range(df.shape[0]):
    df['rest_number'].iloc[i] = data_grouped[data_grouped.index == df['City'].iloc[i]]['rest_number']


In [451]:
# уберем ненужные теперь столбцы
df.drop(['cuisines', 'price_range', 'reviews', 'Unnamed: 0', 'URL_TA', 'URL_TA_full', 'mean_check', 'Reviews', 'ranking'], axis = 1, inplace=True)

In [452]:
# Посмотрим что получилось
df.sample(2)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,ID_TA,Main,alive,rating,excelent,very_good,average,poor,terrible,time_between,last_time_review,text_reviews,rest_number
38177,id_1738,Athens,[Greek],1740.0,4.0,$*,6.0,d3371337,True,1,4.0,4.0,0.0,0.0,0.0,0.0,28.0,2597.0,"'Very nice', 'Still a favourite!!!'",2460.0
29569,id_1164,Edinburgh,[usual],1165.0,4.5,$*,5.0,d11948944,True,1,4.5,3.0,3.0,0.0,0.0,0.0,51.0,1723.0,"'Small but good', 'Top value'",1785.0


In [453]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      50000 non-null  object 
 1   City               50000 non-null  object 
 2   Cuisine Style      50000 non-null  object 
 3   Ranking            50000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        50000 non-null  object 
 6   Number of Reviews  50000 non-null  object 
 7   ID_TA              50000 non-null  object 
 8   Main               50000 non-null  bool   
 9   alive              50000 non-null  int64  
 10  rating             50000 non-null  float64
 11  excelent           50000 non-null  float64
 12  very_good          50000 non-null  float64
 13  average            50000 non-null  float64
 14  poor               50000 non-null  float64
 15  terrible           50000 non-null  float64
 16  time_between       500

### добавим признаки

In [457]:
# обработаем Restaurant_id (является сетевым или нет) сделаем из него бинарный признак
rest_id = df['Restaurant_id'].value_counts()
df['net_rest'] = df['Restaurant_id'].apply(lambda x: 1 if rest_id[rest_id.index == x][0] > 1 else 0)

# Добавим столбец, относится город к столице или нет
capitals = ['Paris', 'Stockholm', 'London', 'Berlin', 'Bratislava', 'Vienna', 'Rome', 'Madrid', 'Dublin', 'Brussels',
            'Warsaw', 'Budapest', 'Copenhagen', 'Amsterdam', 'Lisbon', 'Prague', 'Oslo','Helsinki', 'Edinburgh', 
            'Ljubljana', 'Athens','Luxembourg']
df['is_capital'] = df['City'].apply(lambda x: 1 if x in capitals else 0)

# добавим столбец с кол-вом кухонь
df['cuisines_number'] = df['Cuisine Style'].apply(lambda x: len(x))

# Добавим население из Google (честно заимствовано у коллег)
population = {
    'London': 8908081,
    'Paris': 2240621,
    'Madrid': 3155360,
    'Barcelona': 1593075,
    'Berlin': 3326002,
    'Milan': 1331586,
    'Rome': 2870493,
    'Prague': 1272690,
    'Lisbon': 547733,
    'Vienna': 1765649,
    'Amsterdam': 825080,
    'Brussels': 144784,
    'Hamburg': 1718187,
    'Munich': 1364920,
    'Lyon': 496343,
    'Stockholm': 1981263,
    'Budapest': 1744665,
    'Warsaw': 1720398,
    'Dublin': 506211 ,
    'Copenhagen': 1246611,
    'Athens': 3168846,
    'Edinburgh': 476100,
    'Zurich': 402275,
    'Oporto': 221800,
    'Geneva': 196150,
    'Krakow': 756183,
    'Oslo': 673469,
    'Helsinki': 574579,
    'Bratislava': 413192,
    'Luxembourg': 576249,
    'Ljubljana': 277554
}
df['population'] = df['City'].map(population)

# площадь города, квадратных километров 
square = {
    'London': 1572,
    'Paris': 105.4,
    'Madrid': 607,
    'Barcelona': 100.4,
    'Berlin': 891.68,
    'Milan': 181.67,
    'Rome': 1287.36,
    'Prague': 500,
    'Lisbon': 100.05,
    'Vienna': 414.75,
    'Amsterdam': 219.4,
    'Brussels': 32.61,
    'Hamburg': 755.09,
    'Munich': 310.71,
    'Lyon': 47.87,
    'Stockholm': 188,
    'Budapest': 525.14,
    'Warsaw': 517,
    'Dublin':  318,
    'Copenhagen': 86.4,
    'Athens': 412,
    'Edinburgh': 118,
    'Zurich': 91.88,
    'Oporto': 41.66,
    'Geneva': 15.93,
    'Krakow': 327,
    'Oslo': 454,
    'Helsinki': 715.48,
    'Bratislava': 368,
    'Luxembourg': 2586.4,
    'Ljubljana': 163.8
}
df['city_square'] = df['City'].map(square)

In [461]:
# С помощью библиотеки SentimentIntensityAnalyzer добавим признаки - с какой вероятностью к какому отзыву относится сам отзыв
# Определение тональности отзывов. По стуи вероятность отнесения к позитивному, негативному или нейтральному отзыву

df['negative'] = df['text_reviews'].apply(lambda x: sid.polarity_scores(str(x))['neg']) 

df['neutra'] = df['text_reviews'].apply(lambda x: sid.polarity_scores(str(x))['neu'])

df['positive'] = df['text_reviews'].apply(lambda x: sid.polarity_scores(str(x))['pos'])

df['compound'] = df['text_reviews'].apply(lambda x: sid.polarity_scores(str(x))['compound'])

### на этом пока остановимся. Дальнейшая генерация признаков, при более тщательном рассмотрениии каждого в EDA

In [465]:
df.to_csv(r'all_in.csv', index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      50000 non-null  object 
 1   City               50000 non-null  object 
 2   Cuisine Style      50000 non-null  object 
 3   Ranking            50000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        50000 non-null  object 
 6   Number of Reviews  50000 non-null  object 
 7   ID_TA              50000 non-null  object 
 8   Main               50000 non-null  bool   
 9   alive              50000 non-null  int64  
 10  rating             50000 non-null  float64
 11  excelent           50000 non-null  float64
 12  very_good          50000 non-null  float64
 13  average            50000 non-null  float64
 14  poor               50000 non-null  float64
 15  terrible           50000 non-null  float64
 16  time_between       500